In [11]:
import pandas as pd
from nltk import sent_tokenize
from tqdm import tqdm
tqdm.pandas()
import numpy as np

In [2]:
filepath = './Statements_to_be_analyzed.csv'

In [5]:
doc_df = pd.read_csv(filepath)
doc_df

,meeting_date,document
0,1994-02-04,Chairman Alan Greenspan announced today that t...
1,1994-03-22,Chairman Alan Greenspan announced today that t...
2,1994-04-18,Chairman Alan Greenspan announced today that t...
3,1994-05-17,The Federal Reserve today announced two action...
4,1994-08-16,The Federal Reserve announced today the follow...
...,...,...
208,2022-01-26,Indicators of economic activity and employment...
209,2022-03-16,Indicators of economic activity and employment...
210,2022-05-04,Although overall economic activity edged down ...
211,2022-06-15,Overall economic activity appears to have pick...


In [8]:
def extract_sentences(doc):
    return sent_tokenize(doc)

dfs = []
for _, row in tqdm(doc_df.iterrows()):
    sentences = extract_sentences(row['document'])

    one_doc_sentences_df = pd.DataFrame(sentences, columns=['sentence'])
    one_doc_sentences_df['meeting_date'] = row['meeting_date']
    one_doc_sentences_df['sentence_index'] = one_doc_sentences_df.index
    dfs.append(one_doc_sentences_df)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

print('Number of sentences: {}'.format(len(df)))
df

213it [00:00, 306.16it/s]


Number of sentences: 2903


,sentence,meeting_date,sentence_index
0,Chairman Alan Greenspan announced today that t...,1994-02-04,0
1,The action is expected to be associated with a...,1994-02-04,1
2,The decision was taken to move toward a less a...,1994-02-04,2
3,Chairman Greenspan decided to announce this ac...,1994-02-04,3
4,Chairman Alan Greenspan announced today that t...,1994-03-22,0
...,...,...,...
2898,The Committee is strongly committed to returni...,2022-07-27,9
2899,In assessing the appropriate stance of monetar...,2022-07-27,10
2900,The Committee would be prepared to adjust the ...,2022-07-27,11
2901,The Committee's assessments will take into acc...,2022-07-27,12


Feature extraction

In [9]:
from transformers import pipeline
nlp_features = pipeline('feature-extraction', model="nlpaueb/sec-bert-base")

C:\Users\Jihye Park\anaconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at nlpaueb/sec-bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model tha

In [ ]:
def get_feature(sentence):
    return np.squeeze(nlp_features(sentence)).mean(0) # Get the average values of the embedding layers

df['feature'] = df['sentence'].progress_apply(lambda x: get_feature(x))

 75%|██████████████████████████████████████████████████████████▎                   | 2168/2903 [06:19<01:48,  6.74it/s]